In [1]:
import os

import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
f = gzip.GzipFile(r"../../Data/rBergomiTrainSet.txt.gz", "r")
dat=np.load(f)
xx=dat[:,:4]
yy=dat[:,4:]
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

In [3]:
yy = yy.reshape(-1,8,11)
xx = np.repeat(xx, 8*11,axis=0)

def get_x_index(a,x_dim):
    index_array = np.arange(a.shape[0])
    
    return index_array // x_dim

def get_y_index(a,y_dim):
    index_array = np.arange(a.shape[0])
    
    return index_array % y_dim

def append_and_expand(a,x,y):
    # use choose and where !
    x_index = get_x_index(a,len(x))
    y_index = get_y_index(a,len(y))
    
    x_added = np.choose(x_index,)